In [1]:
import pandas as pd
import numpy as np
import holidays
from datetime import timedelta

# 데이터 파일 경로
file_paths = [
    r"C:\Users\james\DATA_LAB\KTX_Forecasting\data\(간선)수송-운행일-주운행(201501-202305).xlsx",
    r"C:\Users\james\DATA_LAB\KTX_Forecasting\data\(간선)수송-운행일-주운행(202305-202403).xlsx",
    r"C:\Users\james\DATA_LAB\KTX_Forecasting\data\(간선)시종착역별 열차운행(201501-202305).xlsx",
    r"C:\Users\james\DATA_LAB\KTX_Forecasting\data\(간선)시종착역별 열차운행(202305-202403).xlsx",
    r"C:\Users\james\DATA_LAB\KTX_Forecasting\data\korea_cpi_data.xlsx",
    r"C:\Users\james\DATA_LAB\KTX_Forecasting\data\covid_data_final.xlsx"
]

# 데이터 로드 (지정된 행을 열 이름으로 사용)
data_frames = [
    pd.read_excel(file_paths[0], header=5),
    pd.read_excel(file_paths[1], header=5),
    pd.read_excel(file_paths[2], header=8),
    pd.read_excel(file_paths[3], header=8)
]

# cpi 데이터 로드
df_cpi = pd.read_excel(file_paths[4])

# '발표일' 및 '적용일' 열을 문자열로 변환
df_cpi['발표일'] = df_cpi['발표일'].astype(str)
df_cpi['적용일'] = df_cpi['적용일'].astype(str)

# 적용년월 변수 추가
df_cpi['적용일'] = df_cpi['적용일'].str.replace('월', '').str.zfill(2)
df_cpi['적용년월'] = df_cpi['발표일'].str[:4] + '-' + df_cpi['적용일']

# 상치 부합여부 변수 추가(0이면 예상치 부합, 양수면 예상치 상회, 음수면 예상치 하회)
df_cpi['CPI예상치부합여부'] = df_cpi['실제'] - df_cpi['예측']

# 코로나 데이터 로드(2020-01-20~2023-08-31)
df_covid = pd.read_excel(file_paths[5])

# 날짜 형식 변환 및 월단위로 추출
df_covid['날짜'] = pd.to_datetime(df_covid['날짜'], format='%Y-%m-%d')
df_covid['년월'] = df_covid['날짜'].dt.to_period('M')

# 월별 집계
df_covid_monthly = df_covid.groupby('년월').agg({
    '사망자 수': 'sum',
    '확진자 수': 'sum'
}).reset_index()

# 치사율 계산
df_covid_monthly['치사율'] = df_covid_monthly['사망자 수'] / df_covid_monthly['확진자 수'] * 100

# 데이터 결합
df_demand = pd.concat([data_frames[0], data_frames[1]], ignore_index=True)
df_info = pd.concat([data_frames[2], data_frames[3]], ignore_index=True)

# 데이터 필터링 (역무열차종: KTX, 주운행선: '경부선', '경전선', '동해선', '전라선', '호남선')
df_demand = df_demand[df_demand['역무열차종'].apply(lambda x: x[:3] == 'KTX')].reset_index(drop=True)
df_info = df_info[df_info['역무열차종'].apply(lambda x: x[:3] == 'KTX')].reset_index(drop=True)
df_demand = df_demand[df_demand['주운행선'].isin(['경부선', '경전선', '동해선', '전라선', '호남선'])].reset_index(drop=True)
df_info = df_info[df_info['주운행선'].isin(['경부선', '경전선', '동해선', '전라선', '호남선'])].reset_index(drop=True)

# 불필요한 열 삭제
df_demand.drop(columns=['Unnamed: 1', '운행년도', '운행년월', '운행요일구분', '역무열차종', '메트릭'], inplace=True)
df_info.drop(columns=['상행하행구분', '역무열차종', '운행요일구분', '메트릭'], inplace=True)

# 변수 생성
df_demand['1인당수입율'] = df_demand.apply(lambda row: row['승차수입금액'] / row['승차인원수'] if row['승차인원수'] > 0 else 0, axis=1)
df_demand['공급대비승차율'] = df_demand.apply(lambda row: row['승차인원수'] / row['공급좌석합계수'] if row['공급좌석합계수'] > 0 else 0, axis=1)
df_demand['운행대비고객이동'] = df_demand.apply(lambda row: row['좌석거리'] / row['승차연인거리'] if row['승차연인거리'] > 0 else 0, axis=1)

# 날짜 형식 통일
df_demand['운행일자'] = pd.to_datetime(df_demand['운행일자'], format='%Y년 %m월 %d일')
df_info['운행일자'] = pd.to_datetime(df_info['운행일자'], format='%Y년 %m월 %d일')

# 요일 변수 생성
df_demand['요일'] = df_demand['운행일자'].dt.weekday
df_info['요일'] = df_info['운행일자'].dt.weekday
weekday_list = ['월', '화', '수', '목', '금', '토', '일']
df_demand['요일'] = df_demand['요일'].apply(lambda x: weekday_list[x])
df_info['요일'] = df_info['요일'].apply(lambda x: weekday_list[x])

# 주말/주중 변수 생성 (금요일 포함)
df_demand['주말여부'] = df_demand['요일'].apply(lambda x: '주말' if x in ['금', '토', '일'] else '주중')
df_info['주말여부'] = df_info['요일'].apply(lambda x: '주말' if x in ['금', '토', '일'] else '주중')

# 공휴일 변수 생성
kr_holidays = holidays.KR()

def is_holiday(date):
    return date in kr_holidays

df_demand['공휴일여부'] = df_demand['운행일자'].apply(lambda x: '공휴일' if is_holiday(x) else '평일')

# 명절 날짜 설정
seollal_dates = pd.to_datetime(['2015-02-19', '2016-02-08', '2017-01-28', '2018-02-16', '2019-02-05', '2020-01-25', '2021-02-12', '2022-02-01', '2023-01-22', '2024-02-10'])
chuseok_dates = pd.to_datetime(['2015-09-27', '2016-09-15', '2017-10-04', '2018-09-24', '2019-09-13', '2020-10-01', '2021-09-21', '2022-09-10', '2023-09-29', '2024-09-17'])

# 명절 연휴 포함 (설날, 설날 전날, 설날 다음날 / 추석, 추석 전날, 추석 다음날)
seollal_holidays = seollal_dates.union(seollal_dates - timedelta(days=1)).union(seollal_dates + timedelta(days=1))
chuseok_holidays = chuseok_dates.union(chuseok_dates - timedelta(days=1)).union(chuseok_dates + timedelta(days=1))

# 명절 변수 생성
df_demand['명절여부'] = df_demand['운행일자'].apply(lambda x: '명절' if x in seollal_holidays or x in chuseok_holidays else '비명절')

# 일자별 데이터를 기반으로 월별 변수 계산
all_dates = pd.date_range(start=df_demand['운행일자'].min(), end=df_demand['운행일자'].max(), freq='D')
date_info = pd.DataFrame(all_dates, columns=['운행일자'])

date_info['요일'] = date_info['운행일자'].dt.weekday
date_info['주말여부'] = date_info['요일'].apply(lambda x: '주말' if x in [4, 5, 6] else '주중')  # 금, 토, 일은 주말로 설정
date_info['공휴일여부'] = date_info['운행일자'].apply(lambda x: '공휴일' if is_holiday(x) else '평일')
date_info['명절여부'] = date_info['운행일자'].apply(lambda x: '명절' if x in seollal_holidays or x in chuseok_holidays else '비명절')
date_info['운행년월'] = date_info['운행일자'].dt.to_period('M')

monthly_info = date_info.groupby('운행년월').agg({
    '주말여부': lambda x: (x == '주말').sum(),
    '공휴일여부': lambda x: (x == '공휴일').sum(),
    '명절여부': lambda x: (x == '명절').sum(),
    '요일': 'count'  # 총 일수 계산
}).rename(columns={'주말여부': '주말수', '공휴일여부': '공휴일수', '명절여부': '명절수', '요일': '총일수'}).reset_index()

# 월별 집계
df_demand['운행년월'] = df_demand['운행일자'].dt.to_period('M')
df_info['운행년월'] = df_info['운행일자'].dt.to_period('M')

# 공급좌석합계수와 승차연인거리가 0인 경우를 제거
df_demand_cleaned = df_demand[(df_demand['공급좌석합계수'] > 0) & (df_demand['승차연인거리'] > 0)]

df_demand_monthly = df_demand_cleaned.groupby(['주운행선', '운행년월']).agg({
    '공급차량수': 'sum',
    '공급좌석합계수': 'sum',
    '승차수입금액': 'sum',
    '승차인원수': 'sum',
    '승차연인거리': 'sum',
    '좌석거리': 'sum',
    '1인당수입율': 'mean',
    '공급대비승차율': lambda x: x[x > 0].mean(),
    '운행대비고객이동': lambda x: x[x > 0].mean()
}).reset_index()

# df_info에서 날짜 열 제거
df_info_numeric = df_info.drop(columns=['운행일자'])

# 월별 집계 (df_info)
df_info_monthly = df_info_numeric.groupby(['주운행선', '운행년월']).sum(numeric_only=True).reset_index()

# 월별 데이터를 하나의 데이터프레임으로 결합
df_ktx = pd.merge(df_demand_monthly, df_info_monthly, on=['주운행선', '운행년월'], how='inner')

# '평일수' 계산
df_ktx = pd.merge(df_ktx, monthly_info, on='운행년월', how='left')
df_ktx['평일수'] = df_ktx['총일수'] - df_ktx['주말수']

# 불필요한 열 삭제
df_ktx.drop(columns=['총일수'], inplace=True)

# 운행년월과 적용년월이 같은 경우 결합
df_cpi['적용년월'] = df_cpi['적용년월'].apply(lambda x: pd.Period(x, freq='M'))
df_ktx['운행년월'] = df_ktx['운행년월'].apply(lambda x: pd.Period(x, freq='M'))

# 두 데이터프레임 결합
df_ktx = pd.merge(df_ktx, df_cpi[['적용년월', 'CPI예상치부합여부']], left_on='운행년월', right_on='적용년월', how='left')

# 필요 없는 열 삭제
df_ktx.drop(columns=['적용년월'], inplace=True)

# 코로나 데이터 결합
df_covid_monthly['년월'] = df_covid_monthly['년월'].apply(lambda x: pd.Period(str(x), freq='M'))
df_ktx['운행년월'] = df_ktx['운행년월'].apply(lambda x: pd.Period(str(x), freq='M'))

# 두 데이터프레임 결합
df_ktx = pd.merge(df_ktx, df_covid_monthly, left_on='운행년월', right_on='년월', how='left')

# 필요 없는 열 삭제
df_ktx.drop(columns=['년월'], inplace=True)

# 결측치 처리(코로나 지표의 결측치 0으로 대체)
df_ktx = df_ktx.fillna(0)

# 결합된 데이터 확인
df_ktx.head()

# 데이터 저장 (엑셀 파일)
df_ktx.to_excel(r"C:\Users\james\DATA_LAB\KTX_Forecasting\data\ktx_20240602_data.xlsx", index=False)

C:\Users\james\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\james\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\james\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
